In [1]:
import csv
import pandas as pd
import numpy as np
import pymysql.cursors

database = "healthCC"

def create_database(db_file):
    try:
        conn = pymysql.connect(host='localhost',
                         user='root',
                         password='root')
        conn.cursor().execute('CREATE DATABASE IF NOT EXISTS '+db_file)
        
        return True
    except pymysql.InternalError as e:
        return False
    
def create_connection(db_file):
    """ create a database connection to the MySQL database
        specified by db_file
    :param db_file: database file
    :return: Connection object or None
    """
    try:
        connection = pymysql.connect(host='localhost',
                             user='root',
                             password='root',
                             db=db_file)
        return connection
    except pymysql.InternalError as e:
        return False

def create_table(conn, create_table_sql):
    """ create a table from the create_table_sql statement
    :param conn: Connection object
    :param create_table_sql: a CREATE TABLE statement
    :return:
    """
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except pymysql.InternalError as e:
        print(e)

## Create table queries
create_hospital_table = """ CREATE TABLE IF NOT EXISTS hospitals (
                                        provider_id INTEGER PRIMARY KEY,
                                        name TEXT NOT NULL,
                                        type TEXT NOT NULL,
                                        phone TEXT,
                                        url TEXT,
                                        rating INTEGER
                                    ) ENGINE=InnoDB DEFAULT CHARSET=utf8 COLLATE=utf8_bin; """
create_address_table = """ CREATE TABLE IF NOT EXISTS address (
                                        id INTEGER PRIMARY KEY NOT NULL AUTO_INCREMENT,
                                        address TEXT NOT NULL,
                                        state TEXT,
                                        city TEXT,
                                        zip TEXT NOT NULL,
                                        county TEXT
                                    )ENGINE=InnoDB DEFAULT CHARSET=utf8 COLLATE=utf8_bin
                                    AUTO_INCREMENT=1; """
create_ownership_type_table = """ CREATE TABLE IF NOT EXISTS ownership_type (
                                        id INTEGER PRIMARY KEY NOT NULL AUTO_INCREMENT,
                                        ow_type TEXT NOT NULL
                                    ) ENGINE=InnoDB DEFAULT CHARSET=utf8 COLLATE=utf8_bin
                                    AUTO_INCREMENT=1; """
create_hospital_address_table = """ CREATE TABLE IF NOT EXISTS hospital_address (
                                        hospital_id INTEGER NOT NULL,
                                        address_id INTEGER NOT NULL,
                                        PRIMARY KEY (hospital_id, address_id)
                                    ) ENGINE=InnoDB DEFAULT CHARSET=utf8 COLLATE=utf8_bin; """
create_hospital_ownership_table = """ CREATE TABLE IF NOT EXISTS hospital_ownership (
                                        hospital_id INTEGER NOT NULL,
                                        ownership_id INTEGER NOT NULL,
                                        PRIMARY KEY (hospital_id, ownership_id)
                                    ) ENGINE=InnoDB DEFAULT CHARSET=utf8 COLLATE=utf8_bin; """
create_hospital_comparison_table = """ CREATE TABLE IF NOT EXISTS hospital_comparison (
                                        hospital_id INTEGER NOT NULL PRIMARY KEY,
                                        emergency_services INTEGER,
                                        ehr INTEGER, 
                                        mortality INTEGER, 
                                        safety INTEGER, 
                                        readmission INTEGER, 
                                        patient_experience INTEGER,
                                        effectiveness INTEGER,
                                        timeliness INTEGER,
                                        medical_imaging INTEGER
                                    ) ENGINE=InnoDB DEFAULT CHARSET=utf8 COLLATE=utf8_bin """
######## hcahps
create_hcahps_measure_info_table = """ CREATE TABLE IF NOT EXISTS hcahps_measure_info (
                                        id INTEGER NOT NULL PRIMARY KEY AUTO_INCREMENT,
                                        question TEXT NOT NULL,
                                        response TEXT NOT NULL,
                                        measure_code TEXT NOT NULL,
                                        measure_start_date DATE,
                                        measure_end_date DATE
                                    ) ENGINE=InnoDB DEFAULT CHARSET=utf8 COLLATE=utf8_bin
                                    AUTO_INCREMENT=1; """

create_hospitals_hcahps_qsn_percent_table = """ CREATE TABLE IF NOT EXISTS hospitals_hcahps_qsn_percent (
                                        hospital_id INTEGER NOT NULL,
                                        question_measure_id INTEGER NOT NULL,
                                        percent INTEGER,
                                        star_rating INTEGER,
                                        linear_mean_score INTEGER,
                                        completed_surveys TEXT,
                                        survey_footnote_id INT,
                                        PRIMARY KEY (hospital_id, question_measure_id)
                                    ) ENGINE=InnoDB DEFAULT CHARSET=utf8 COLLATE=utf8_bin; """

create_hcahps_measure_footnotes_table = """ CREATE TABLE IF NOT EXISTS hcahps_measure_footnotes (
                                        id INTEGER NOT NULL PRIMARY KEY,
                                        footnote TEXT NOT NULL
                                    ) ENGINE=InnoDB DEFAULT CHARSET=utf8 COLLATE=utf8_bin; """

### Spending per ben table
create_hospital_spending_per_ben_table = """ CREATE TABLE IF NOT EXISTS hospital_spending_per_ben (
                                        hospital_id INTEGER NOT NULL,
                                        spending_measure_id INTEGER NOT NULL,
                                        score FLOAT,
                                        PRIMARY KEY (hospital_id, spending_measure_id)
                                    ) ENGINE=InnoDB DEFAULT CHARSET=utf8 COLLATE=utf8_bin; """

create_spending_measure_info_table = """ CREATE TABLE IF NOT EXISTS spending_measure_info (
                                        id INTEGER NOT NULL PRIMARY KEY AUTO_INCREMENT,
                                        measure_code TEXT NOT NULL,
                                        national_score FLOAT,
                                        national_median FLOAT,
                                        measure_start_date DATE,
                                        measure_end_date DATE
                                    ) ENGINE=InnoDB DEFAULT CHARSET=utf8 COLLATE=utf8_bin
                                     AUTO_INCREMENT=1; """

#### Spending by Claim
create_hospital_spending_per_claim_table = """ CREATE TABLE IF NOT EXISTS hospital_spending_per_claim (
                                        hospital_id INTEGER NOT NULL,
                                        claim_type INTEGER NOT NULL,
                                        claim_period_id INTEGER NOT NULL,
                                        avg_spending_hospital INTEGER,
                                        avg_spending_state INTEGER,
                                        percent_spending_hospital FLOAT,
                                        percent_spending_state FLOAT,
                                        percent_spending_nation FLOAT,
                                        measure_start_date DATE,
                                        measure_end_date DATE,
                                        PRIMARY KEY (hospital_id, claim_type, claim_period_id)
                                    ) ENGINE=InnoDB DEFAULT CHARSET=utf8 COLLATE=utf8_bin; """

create_claim_types_table = """ CREATE TABLE IF NOT EXISTS claim_types (
                                        id INTEGER NOT NULL PRIMARY KEY AUTO_INCREMENT,
                                        type_name TEXT
                                    ) ENGINE=InnoDB DEFAULT CHARSET=utf8 COLLATE=utf8_bin
                                    AUTO_INCREMENT=1; """

create_claim_time_period_table = """ CREATE TABLE IF NOT EXISTS claim_time_period (
                                        id INTEGER NOT NULL PRIMARY KEY AUTO_INCREMENT,
                                        period_name TEXT
                                    ) ENGINE=InnoDB DEFAULT CHARSET=utf8 COLLATE=utf8_bin
                                    AUTO_INCREMENT=1; """

#### Strucural Measures
create_hospitals_structural_measure_table = """ CREATE TABLE IF NOT EXISTS hospitals_structural_measure (
                                        hospital_id INTEGER NOT NULL,
                                        measure_type_id INTEGER NOT NULL,
                                        measure_response INTEGER,
                                        PRIMARY KEY (hospital_id, measure_type_id)
                                    ) ENGINE=InnoDB DEFAULT CHARSET=utf8 COLLATE=utf8_bin; """

create_structural_measure_info_table = """ CREATE TABLE IF NOT EXISTS structural_measure_info (
                                        id INTEGER NOT NULL PRIMARY KEY AUTO_INCREMENT,
                                        type_name TEXT,
                                        measure_start_date DATE,
                                        measure_end_date DATE
                                    ) ENGINE=InnoDB DEFAULT CHARSET=utf8 COLLATE=utf8_bin
                                    AUTO_INCREMENT=1; """

#### Mortality
create_mortality_measure_info_table = """ CREATE TABLE IF NOT EXISTS mortality_measure_info (
                                        id INTEGER NOT NULL PRIMARY KEY AUTO_INCREMENT,
                                        type_name TEXT,
                                        mortality_code TEXT,
                                        national_rate FLOAT,
                                        measure_start_date DATE,
                                        measure_end_date DATE
                                    ) ENGINE=InnoDB DEFAULT CHARSET=utf8 COLLATE=utf8_bin
                                    AUTO_INCREMENT=1; """

create_hospitals_mortality_table = """ CREATE TABLE IF NOT EXISTS hospitals_mortality (
                                        hospital_id INTEGER NOT NULL,
                                        mortality_id INTEGER NOT NULL,
                                        national_comparison INTEGER,
                                        denominator INTEGER,
                                        score FLOAT,
                                        lower_est FLOAT,
                                        higher_est FLOAT,
                                        PRIMARY KEY (hospital_id, mortality_id)
                                    ) ENGINE=InnoDB DEFAULT CHARSET=utf8 COLLATE=utf8_bin; """

create_mortality_national_comparison_type_table = """ CREATE TABLE IF NOT EXISTS mortality_national_comparison_type (
                                        id INTEGER NOT NULL PRIMARY KEY AUTO_INCREMENT,
                                        comparison_name TEXT
                                    ) ENGINE=InnoDB DEFAULT CHARSET=utf8 COLLATE=utf8_bin
                                    AUTO_INCREMENT=1; """

###### Payment and Value Care
create_payment_measure_info_table = """ CREATE TABLE IF NOT EXISTS payment_measure_info (
                                        id INTEGER NOT NULL PRIMARY KEY AUTO_INCREMENT,
                                        type_name TEXT,
                                        measure_code TEXT,
                                        measure_start_date DATE,
                                        measure_end_date DATE
                                    ) ENGINE=InnoDB DEFAULT CHARSET=utf8 COLLATE=utf8_bin
                                    AUTO_INCREMENT=1; """

create_payment_measure_category_table = """ CREATE TABLE IF NOT EXISTS payment_measure_category (
                                        id INTEGER NOT NULL PRIMARY KEY AUTO_INCREMENT,
                                        category_name TEXT
                                    ) ENGINE=InnoDB DEFAULT CHARSET=utf8 COLLATE=utf8_bin
                                    AUTO_INCREMENT=1; """

create_value_measure_info_table = """ CREATE TABLE IF NOT EXISTS value_measure_info (
                                        id INTEGER NOT NULL PRIMARY KEY AUTO_INCREMENT,
                                        type_name TEXT,
                                        measure_code TEXT,
                                        measure_start_date DATE,
                                        measure_end_date DATE
                                    ) ENGINE=InnoDB DEFAULT CHARSET=utf8 COLLATE=utf8_bin
                                    AUTO_INCREMENT=1; """

create_value_measure_category_table = """ CREATE TABLE IF NOT EXISTS value_measure_category (
                                        id INTEGER NOT NULL PRIMARY KEY AUTO_INCREMENT,
                                        category_name TEXT
                                    ) ENGINE=InnoDB DEFAULT CHARSET=utf8 COLLATE=utf8_bin
                                    AUTO_INCREMENT=1; """

create_hospitals_value_measure_table = """ CREATE TABLE IF NOT EXISTS hospitals_value_measure (
                                        hospital_id INTEGER NOT NULL,
                                        value_measure_id INTEGER NOT NULL,
                                        value_category INTEGER,
                                        PRIMARY KEY (hospital_id, value_measure_id)
                                    ) ENGINE=InnoDB DEFAULT CHARSET=utf8 COLLATE=utf8_bin; """

create_hospitals_payment_measure_table = """ CREATE TABLE IF NOT EXISTS hospitals_payment_measure (
                                        hospital_id INTEGER NOT NULL,
                                        payment_measure_id INTEGER NOT NULL,
                                        payment_category_id INTEGER,
                                        denominator INTEGER,
                                        payment INTEGER,
                                        lower_est INTEGER,
                                        highesr_est INTEGER,
                                        PRIMARY KEY (hospital_id, payment_measure_id)
                                    ) ENGINE=InnoDB DEFAULT CHARSET=utf8 COLLATE=utf8_bin; """

##### YELP

create_yelp_hosp_info_table = """ CREATE TABLE IF NOT EXISTS yelp_hosp_info (
                                        hospital_id INTEGER NOT NULL PRIMARY KEY,
                                        yelp_id TEXT,
                                        yelp_url TEXT,
                                        rating FLOAT,
                                        total_reviews INTEGER,
                                        accepts_credit_card INTEGER,
                                        accepts_insurance INTEGER,
                                        gender_neutral_restrooms INTEGER
                                    ) ENGINE=InnoDB DEFAULT CHARSET=utf8 COLLATE=utf8_bin; """

create_yelp_review_table = """ CREATE TABLE IF NOT EXISTS yelp_review (
                                        id INTEGER NOT NULL PRIMARY KEY AUTO_INCREMENT,
                                        hospital_id INTEGER NOT NULL,
                                        review_score FLOAT,
                                        review_date DATE,
                                        reviewed_user_id INTEGER,
                                        userful_score INTEGER,
                                        cool_score INTEGER,
                                        funny_score INTEGER,
                                        review_desc TEXT
                                    ) ENGINE=InnoDB DEFAULT CHARSET=utf8 COLLATE=utf8_bin
                                    AUTO_INCREMENT=1; """

create_yelp_user_info_table = """ CREATE TABLE IF NOT EXISTS yelp_user (
                                        id INTEGER NOT NULL PRIMARY KEY AUTO_INCREMENT,
                                        username TEXT,
                                        user_state TEXT,
                                        user_city TEXT
                                    ) ENGINE=InnoDB DEFAULT CHARSET=utf8 COLLATE=utf8_bin
                                    AUTO_INCREMENT=1; """

create_yelp_user_rating_table = """ CREATE TABLE IF NOT EXISTS yelp_user_rating (
                                        yelp_review_id INTEGER NOT NULL,
                                        yelp_user_id INTEGER NOT NULL,
                                        PRIMARY KEY (yelp_review_id, yelp_user_id)
                                    ) ENGINE=InnoDB DEFAULT CHARSET=utf8 COLLATE=utf8_bin; """

create_yelp_tags_table = """ CREATE TABLE IF NOT EXISTS yelp_tags (
                                        id INTEGER NOT NULL PRIMARY KEY AUTO_INCREMENT,
                                        tag_name TEXT
                                    ) ENGINE=InnoDB DEFAULT CHARSET=utf8 COLLATE=utf8_bin
                                    AUTO_INCREMENT=1; """

create_yelp_hospital_tags_table = """ CREATE TABLE IF NOT EXISTS yelp_hospital_tags (
                                        hospital_id INTEGER NOT NULL,
                                        tag_id INTEGER NOT NULL,
                                        PRIMARY KEY (hospital_id, tag_id)
                                    ) ENGINE=InnoDB DEFAULT CHARSET=utf8 COLLATE=utf8_bin; """


# Insert queries

def insert_hospital(cur, hospital_data):
    """ insert into hospital table 
    :param cur: cursor
    :param hospital_data: hospital data
    :return: current inserted row id
    """
    ## 'Provider ID','Hospital Name','Hospital Type', 'Phone Number', 'Hospital overall rating'
    hp_sql = """INSERT IGNORE INTO hospitals(provider_id, name, type, phone, rating)
            VALUES(%s,%s,%s,%s,%s)"""
    hp_data=(int(hospital_data['Provider ID']),
          hospital_data['Hospital Name'],
          hospital_data['Hospital Type'],
          hospital_data['Phone Number'],
          hospital_data['Hospital overall rating'])
    
    cur.execute(hp_sql, hp_data)
    
    return int(hospital_data['Provider ID'])

def insert_address(cur, address_data):
    """ insert into address table 
    :param cur: cursor
    :param address_data: address data
    :return: current inserted row id
    """
    ## 'Address','State','City', 'Zip Code', 'County Name'
    add_sql = """INSERT INTO address(address, state, city, zip, county) 
                VALUES(%s,%s,%s,%s,%s)"""
    add_data = (address_data['Address'],
               address_data['State'],
               address_data['City'],
               address_data['ZIP Code'],
               address_data['County Name'])

    cur.execute(add_sql, add_data)
    
    return cur.lastrowid

def insert_hospital_address(conn, hospital_id, address_id):
    """ insert into hospital_address table 
    :param cur: cursor
    :param hospital_id: hospital id
    :param address_id: address id
    :return: current inserted row id
    """
    ## 'Hospital Id','Address Id'
    hp_add_sql = """INSERT IGNORE INTO hospital_address(hospital_id, address_id) 
                    VALUES(%s,%s)""" 
    hp_add_data = (int(hospital_id), int(address_id))
    
    cur.execute(hp_add_sql, hp_add_data)
    
    return cur.lastrowid

def insert_ownership_type(cur, ownership_data):
    """ insert into ownership type table 
    :param cur: cursor
    :param ownership_data: ownership data
    :return: current inserted row id
    """
    ## 'Ownership Type'
    select_data = (ownership_data,)
    cur.execute("""SELECT * FROM ownership_type WHERE ow_type=%s""", select_data)
    entry = cur.fetchone()

    if entry is None:
        insert_data = (ownership_data,)
        cur.execute("""INSERT INTO ownership_type(ow_type)
                        VALUES(%s)""", insert_data)
        lasrowid=cur.lastrowid
    else:
        lasrowid=entry[0]
    
    return lasrowid

def insert_hospital_ownership(cur, hospital_id, ow_id):
    """ insert into hospital_ownership table 
    :param cur: cursor
    :param hospital_id: hospital id
    :param ownership_id: ownership_id
    :return: current inserted row id
    """
    ## insert into hospital_ownership
    hp_ow_sql = """INSERT IGNORE INTO hospital_ownership(hospital_id, ownership_id) 
                    VALUES(%s,%s)""" 
    hp_ow_data = (int(hospital_id), int(ow_id))
    
    cur.execute(hp_ow_sql, hp_ow_data)
    
    return cur.lastrowid

def insert_hospital_comparison(cur, hp_comp_data, hp_id):
    """ insert into hospital_comparison table 
    :param cur: cursor
    :param hp_comp_data: hospital comparison data
    :param hp_id: hospital id 
    :return: current inserted row id
    """
    ## insert into hospital_comparison
    hp_comp_sql = """INSERT INTO hospital_comparison(hospital_id, emergency_services, ehr, mortality, safety, readmission, 
                        patient_experience, effectiveness, timeliness, medical_imaging) 
                    VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"""
    hp_comp_data = (hp_id, 
                    hp_comp_data['Emergency Services'],
                    hp_comp_data['Meets criteria for meaningful use of EHRs'],
                    hp_comp_data['Mortality national comparison'],
                    hp_comp_data['Safety of care national comparison'],
                    hp_comp_data['Readmission national comparison'],
                    hp_comp_data['Patient experience national comparison'],
                    hp_comp_data['Effectiveness of care national comparison'],
                    hp_comp_data['Timeliness of care national comparison'],
                    hp_comp_data['Efficient use of medical imaging national comparison'])
    
    cur.execute(hp_comp_sql, hp_comp_data)
    
    return cur.lastrowid

# create database
res = create_database(database)
# create a database connection
if res==True:
    conn = create_connection(database)

    if conn is not None:
        # create all tables
        tableList = (create_hospital_table, create_address_table, create_ownership_type_table, 
                     create_hospital_address_table, create_hospital_ownership_table, create_hospital_comparison_table,
                    create_hcahps_measure_info_table, create_hospitals_hcahps_qsn_percent_table, create_hcahps_measure_footnotes_table,
                    create_hospital_spending_per_ben_table, create_spending_measure_info_table, create_hospital_spending_per_claim_table, 
                    create_claim_types_table, create_claim_time_period_table, create_hospitals_structural_measure_table, create_structural_measure_info_table, 
                    create_mortality_measure_info_table, create_hospitals_mortality_table, create_mortality_national_comparison_type_table,
                    create_payment_measure_info_table, create_payment_measure_category_table, create_value_measure_info_table, 
                    create_value_measure_category_table, create_hospitals_value_measure_table, create_hospitals_payment_measure_table,
                    create_yelp_hosp_info_table, create_yelp_review_table, create_yelp_user_info_table, create_yelp_user_rating_table,
                    create_yelp_tags_table, create_yelp_hospital_tags_table
                    )
        for table in tableList:
            create_table(conn, table)
        #create_table(conn, create_hospital_table)
        #create_table(conn, create_address_table)
        #create_table(conn, create_ownership_type_table)
        #create_table(conn, create_hospital_address_table)
        #create_table(conn, create_hospital_ownership_table)
        #create_table(conn, create_hospital_comparison_table)
    else:
        print("Error! cannot create the database connection.")

    # Commit connection and close cursor and connection
    conn.commit()
    conn.close()

    conn = create_connection(database)


    chunksize = 500
    cur = conn.cursor()
    for df in pd.read_csv('data/hospital_gen_info.csv', chunksize=chunksize, iterator=True):
        #County ## Replace 'nan' = ''
        County_mapping={np.NaN:''}
        df['County Name'].replace(County_mapping, inplace=True)
        # Hospital Type ## Replace 'Acute Care Hospitals'=1, 'Critical Access Hospitals'=2, 'Childrens'=3
        HType_mapping={'Acute Care Hospitals':1, 'Critical Access Hospitals':2, 'Childrens':3}
        df['Hospital Type'].replace(HType_mapping, inplace=True)
        # 'Meets criteria for meaningful use of EHRs' ## Replace Y=1, nan=-100
        EHR_mapping={'Y':1, np.NaN:-100}
        df['Meets criteria for meaningful use of EHRs'].replace(EHR_mapping, inplace=True)

        # 'Emergency Services' ## Replace Yes=1, No=0, 'nan'=-100
        ES_mapping = {'Yes': 1, 'No': 0, np.NaN:-100}
        df['Emergency Services'].replace(ES_mapping, inplace=True)

        # 'Hospital overall rating ## Replace Not Available as -100 
        df['Hospital overall rating'].replace({'Not Available':-100}, inplace=True)

        # Same as the national average = 0, Below the national average = -1, Above the national average = 1, Not Available = -100
        MC_mapping = {'Same as the national average' : 0,
                      'Below the national average' : -1, 
                      'Above the national average' : 1, 
                      'Not Available' : -100 }

        df['Mortality national comparison'].replace(MC_mapping, inplace=True)
        df['Safety of care national comparison'].replace(MC_mapping, inplace=True)
        df['Readmission national comparison'].replace(MC_mapping, inplace=True)
        df['Patient experience national comparison'].replace(MC_mapping, inplace=True)
        df['Effectiveness of care national comparison'].replace(MC_mapping, inplace=True)
        df['Timeliness of care national comparison'].replace(MC_mapping, inplace=True)
        df['Efficient use of medical imaging national comparison'].replace(MC_mapping, inplace=True)

        for index, row in df.iterrows():
            hospital_id = insert_hospital(cur, row)
            address_id = insert_address(cur, row)

            insert_hospital_address(cur, hospital_id, address_id)

            ow_id = insert_ownership_type(cur, row['Hospital Ownership'])
            
            insert_hospital_ownership(cur, hospital_id, ow_id)
            #print(row)
            insert_hospital_comparison(cur, row, hospital_id)
            
    conn.commit()
    conn.close()
       

/home/joshua/anaconda3/envs/healthCC/lib/python3.6/site-packages/pymysql/cursors.py:166: Warning: (1007, "Can't create database 'healthCC'; database exists")
  result = self._query(query)
/home/joshua/anaconda3/envs/healthCC/lib/python3.6/site-packages/pymysql/cursors.py:166: Warning: (1050, "Table 'hospitals' already exists")
  result = self._query(query)
/home/joshua/anaconda3/envs/healthCC/lib/python3.6/site-packages/pymysql/cursors.py:166: Warning: (1050, "Table 'address' already exists")
  result = self._query(query)
/home/joshua/anaconda3/envs/healthCC/lib/python3.6/site-packages/pymysql/cursors.py:166: Warning: (1050, "Table 'ownership_type' already exists")
  result = self._query(query)
/home/joshua/anaconda3/envs/healthCC/lib/python3.6/site-packages/pymysql/cursors.py:166: Warning: (1050, "Table 'hospital_address' already exists")
  result = self._query(query)
/home/joshua/anaconda3/envs/healthCC/lib/python3.6/site-packages/pymysql/cursors.py:166: Warning: (1050, "Table 'hospi